In [0]:
# 1. SETUP: Import the necessary tools 🛠️
from pyspark.sql.functions import (
    col, lit, current_timestamp, trim, to_timestamp, coalesce, sha2, 
    concat_ws, year, when, regexp_replace
)
from delta.tables import DeltaTable
import datetime
import sys
import traceback

# --- CONFIGURATION ---

# 1a. Define File Locations (Paths)
SOURCE_PATH = "abfss://project@scrgvkrmade.dfs.core.windows.net/bronze/ResellerSales/Production.Product/" # Changed path to Product source for clarity
TARGET_PATH = "abfss://project@scrgvkrmade.dfs.core.windows.net/silver/dim/dim_product/"

# 1b. Define the Unique Identifier (Primary Key) - REFLECTS NEW MAPPING
# Keys must be: ProductKey (Source ID), ProductAlternateKey (Product Number), ProductSubcategoryKey
PK_RAW = "ProductKey,ProductAlternateKey,ProductSubcategoryKey"
PRIMARY_KEYS = [c.strip() for c in PK_RAW.split(",") if c.strip()]

# 1c. Define Column Renaming and Mapping (Bronze to Silver)
COLUMN_MAP = {
    "ProductID": "ProductKey", # Source ID -> Surrogate Key (New Mapping)
    "ProductSubcategoryID": "ProductSubcategoryKey", 
    "WeightUnitMeasureCode": "WeightUnitMeasureCode",
    "SizeUnitMeasureCode": "SizeUnitMeasureCode",
    "Name": "EnglishProductName", 
    "ProductNumber": "ProductAlternateKey", # Product Number -> Alternate Key (New Mapping)
    "StandardCost": "StandardCost",
    "FinishedGoodsFlag": "FinishedGoodsFlag",
    "Color": "Color",
    "SafetyStockLevel": "SafetyStockLevel",
    "ReorderPoint": "ReorderPoint",
    "ListPrice": "ListPrice",
    "Size": "Size",
    "Weight": "Weight",
    "DaysToManufacture": "DaysToManufacture",
    "ProductLine": "ProductLine",
    "Class": "Class",
    "Style": "Style",
    "SellStartDate": "StartDate",
    "SellEndDate": "EndDate",
    "ModifiedDate": "ModifiedDate"
}

# 1d. Setup Storage Access (Authentication)
storage_account_name = "scrgvkrmade"
account_key = "E4VB7pXWFXttUWbbSBPY35/Dvsw6Fs6XgIWLTj3lCS6v/jCEow9Uxs+r6Usobhenv14UdWEzb+R8+AStNyS0dg=="
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", account_key)
PK_COLS = PRIMARY_KEYS
print(f"Using Primary Key for MERGE: {PK_COLS}")


# --- 2. READ AND CLEAN THE DATA ---
df_source = (spark.read
                  .option("mergeSchema", "true")
                  .option("recursiveFileLookup", "true")
                  .parquet(SOURCE_PATH))

# 2a. Rename Columns
for source_col, target_col in COLUMN_MAP.items():
    if source_col in df_source.columns:
        df_source = df_source.withColumnRenamed(source_col, target_col)
        
# 2b. Data Cleaning Steps
print("2. Cleaning and preparing data...")
for c in ["EnglishProductName", "Color", "Size", "ProductLine", "Class", "Style"]:
    if c in df_source.columns:
        df_source = df_source.withColumn(c, trim(coalesce(col(c).cast("string"), lit("N/A"))))

df_source = df_source.withColumn("StartDate", to_timestamp(col("StartDate")))
df_source = df_source.withColumn("EndDate", to_timestamp(col("EndDate")))
df_source = df_source.withColumn("ModifiedDate", to_timestamp(col("ModifiedDate")))
df_source = df_source.withColumn("StandardCost", col("StandardCost").cast("decimal(19,4)"))
df_source = df_source.withColumn("ListPrice", col("ListPrice").cast("decimal(19,4)"))
df_source = df_source.withColumn("Weight", col("Weight").cast("decimal(8,2)"))
df_source = df_source.dropDuplicates(PK_COLS)


# --- 3. ADD METADATA AND DERIVED COLUMNS ---
print("3. Adding metadata and required Silver columns...")

# 3a. Audit Columns and Hash
df_source = df_source.withColumn("LoadTS", current_timestamp())
df_source = df_source.withColumn("_year", year(col("ModifiedDate")))
# FIX: Updated hash_cols to use ProductAlternateKey instead of ProductNumber
hash_cols = ["ProductKey", "ProductAlternateKey", "ProductSubcategoryKey", "StandardCost", "ListPrice", "FinishedGoodsFlag", "Color", "Weight", "Size"]
df_source = df_source.withColumn(
    "__row_hash",
    sha2(concat_ws("||", *[coalesce(col(c).cast("string"), lit("")) for c in hash_cols]), 256)
)
df_source = df_source.withColumn("__ingest_ts", current_timestamp())
df_source = df_source.withColumn("__source_path", lit(SOURCE_PATH)) 
df_source = df_source.withColumn("__target_path", lit(TARGET_PATH)) 
df_source = df_source.withColumn("__batch_id", lit("Batch-" + datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')))

# 3b. Key and Placeholder Columns
# FIX: Removed the line setting ProductKey to NULL since it's now mapped from ProductID
df_source = df_source.withColumn("Status", when(col("EndDate").isNull(), lit("Current")).otherwise(lit("Discontinued")))
df_source = df_source.withColumn("SizeRange", lit(None).cast("string")) 
df_source = df_source.withColumn("DealerPrice", lit(None).cast("decimal(19,4)")) 
df_source = df_source.withColumn("ModelName", lit(None).cast("string")) 
df_source = df_source.withColumn("IsCurrent", lit(True)) 
df_source = df_source.withColumn("SpanishProductName", lit(None).cast("string"))
df_source = df_source.withColumn("FrenchProductName", lit(None).cast("string"))
df_source = df_source.withColumn("LargePhoto", lit(None).cast("string"))
df_source = df_source.withColumn("EnglishDescription", lit(None).cast("string"))


# --- 4. LOAD INTO DELTA LAKE USING MERGE ---

# 4a. EXPLICIT ALL_TARGET_COLS List
ALL_TARGET_COLS = [
    # Keys
    "ProductKey", "ProductAlternateKey", "ProductSubcategoryKey",
    
    # Mapped Data
    "WeightUnitMeasureCode", "SizeUnitMeasureCode", "EnglishProductName", "StandardCost", 
    "FinishedGoodsFlag", "Color", "SafetyStockLevel", "ReorderPoint", "ListPrice", 
    "Size", "Weight", "DaysToManufacture", "ProductLine", "Class", "Style", 
    
    # Derived/Placeholder (NULL) Columns
    "SpanishProductName", "FrenchProductName", "SizeRange", "DealerPrice", "ModelName", 
    "LargePhoto", "EnglishDescription", 
    
    # SCD/Status/Date Columns
    "StartDate", "EndDate", "Status", "ModifiedDate", "IsCurrent",
    
    # Audit Columns
    "LoadTS", "__row_hash", "_year", "__ingest_ts", "__source_path", 
    "__target_path", "__batch_id"
]

target_exists = DeltaTable.isDeltaTable(spark, TARGET_PATH)

if not target_exists:
    print(f"\n4. Target table not found. Creating initial table at: {TARGET_PATH}")
    
    # Initial Write: Select only the required columns and save
    df_source.select(*[c for c in ALL_TARGET_COLS if c in df_source.columns]).write \
        .format("delta").mode("overwrite").option("overwriteSchema", "true").partitionBy("_year").save(TARGET_PATH)
    print("Initial table created. ✅")

else:
    print(f"\n4. Target table exists. Performing Incremental MERGE...")
    dt_target = DeltaTable.forPath(spark, TARGET_PATH)
    
    # join_cond is now correct: target.ProductKey, target.ProductAlternateKey, target.ProductSubcategoryKey
    join_cond = " AND ".join([f"target.{c} = source.{c}" for c in PK_COLS])
    change_cond = "target.__row_hash != source.__row_hash"
    
    dt_target.alias("target").merge(df_source.alias("source"), join_cond) \
    .whenMatchedUpdate(
        condition=change_cond,
        set = {
            "StandardCost": "source.StandardCost", 
            "ListPrice": "source.ListPrice",
            "Color": "source.Color",
            "Weight": "source.Weight",
            "Size": "source.Size",
            "ModifiedDate": "source.ModifiedDate",
            "Status": "source.Status",
            "LoadTS": "source.LoadTS",
            "__row_hash": "source.__row_hash",
        }
    ) \
    .whenNotMatchedInsert(
        # --- EXPLICIT INSERT VALUES FOR PRODUCT ---
        values = {
            "ProductKey": "source.ProductKey",
            "ProductAlternateKey": "source.ProductAlternateKey",
            "ProductSubcategoryKey": "source.ProductSubcategoryKey",
            "WeightUnitMeasureCode": "source.WeightUnitMeasureCode",
            "SizeUnitMeasureCode": "source.SizeUnitMeasureCode",
            "EnglishProductName": "source.EnglishProductName",
            "StandardCost": "source.StandardCost",
            "FinishedGoodsFlag": "source.FinishedGoodsFlag",
            "Color": "source.Color",
            "SafetyStockLevel": "source.SafetyStockLevel",
            "ReorderPoint": "source.ReorderPoint",
            "ListPrice": "source.ListPrice",
            "Size": "source.Size",
            "Weight": "source.Weight",
            "DaysToManufacture": "source.DaysToManufacture",
            "ProductLine": "source.ProductLine",
            "Class": "source.Class",
            "Style": "source.Style",
            "StartDate": "source.StartDate",
            "EndDate": "source.EndDate",
            "Status": "source.Status",
            "IsCurrent": "source.IsCurrent",
            "SpanishProductName": "source.SpanishProductName", 
            "FrenchProductName": "source.FrenchProductName", 
            "SizeRange": "source.SizeRange", 
            "DealerPrice": "source.DealerPrice", 
            "ModelName": "source.ModelName", 
            "LargePhoto": "source.LargePhoto", 
            "EnglishDescription": "source.EnglishDescription", 
            "LoadTS": "source.LoadTS",
            "__row_hash": "source.__row_hash",
            "_year": "source._year",
            "__ingest_ts": "source.__ingest_ts",
            "__source_path": "source.__source_path",
            "__target_path": "source.__target_path",
            "__batch_id": "source.__batch_id",
            "ModifiedDate": "source.ModifiedDate"
        }
    ) \
    .execute()
    
    print("MERGE (Incremental Load) complete. ✅")